In [1]:
%matplotlib inline

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root = "data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.liniear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.ReLU()
        )
        
    def forward(self,x):
        x = self.flatten(x)
        logits = self.liniear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [6]:
# loss functionの初期化、定義
loss_fn = nn.CrossEntropyLoss()

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # 予測と損失の計算
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 6.259114  [    0/60000]
loss: 6.157572  [ 6400/60000]
loss: 5.899823  [12800/60000]
loss: 5.726341  [19200/60000]
loss: 5.455176  [25600/60000]
loss: 4.518022  [32000/60000]
loss: 4.874815  [38400/60000]
loss: 4.122265  [44800/60000]
loss: 3.824906  [51200/60000]
loss: 4.552082  [57600/60000]
Test Error: 
 Accuracy: 37.2%, Avg loss: 0.066950 

Epoch 2
-------------------------------
loss: 4.058312  [    0/60000]
loss: 4.439381  [ 6400/60000]
loss: 4.121801  [12800/60000]
loss: 4.329647  [19200/60000]
loss: 4.428805  [25600/60000]
loss: 3.627139  [32000/60000]
loss: 4.383322  [38400/60000]
loss: 3.644675  [44800/60000]
loss: 3.348419  [51200/60000]
loss: 4.097969  [57600/60000]
Test Error: 
 Accuracy: 47.8%, Avg loss: 0.060532 

Epoch 3
-------------------------------
loss: 3.657914  [    0/60000]
loss: 4.073530  [ 6400/60000]
loss: 3.750750  [12800/60000]
loss: 3.835212  [19200/60000]
loss: 3.938102  [25600/60000]
loss: 3.238915  [32000/600